In [1]:
import tensorflow as tf
import numpy as np

In [2]:
with open('../model/Trained_Graphs/merged_frozen_graph.pb', 'rb') as f:
    fileContent = f.read()
graph_def = tf.GraphDef()
graph_def.ParseFromString(fileContent)
tf.import_graph_def(graph_def, input_map=None, return_elements=None, name='', op_dict=None, producer_op_list=None)
graph = tf.get_default_graph()
tensors = [n.name for n in tf.get_default_graph().as_graph_def().node]

In [3]:
wtoidx = np.load("../Dataset/wordmap.npy").tolist()
idxtow = dict(zip(wtoidx.values(), wtoidx.keys()))

with open('../model/Decoder/DecoderOutputs.txt', 'r') as fr:
    outputs= fr.read()
    outputs=outputs.split('\n')[:-1]

In [4]:
def IDs_to_Words(ID_batch):
    return [idxtow[word] for IDs in ID_batch for word in IDs]

In [5]:
def init_caption_generator():
    sess = tf.Session()
    return sess

def generate_caption(sess, image_path):
    in1 = graph.get_tensor_by_name("encoder/InputImage:0")
    out1 = graph.get_tensor_by_name("encoder/Preprocessed_JPG:0")
    feed_dict = {in1: image_path}
    prepro_image = sess.run(out1, feed_dict=feed_dict)
    in2 = graph.get_tensor_by_name("encoder/import/InputImage:0")
    outfinal = graph.get_tensor_by_name("encoder/import/InceptionV4/Logits/AvgPool_1a/AvgPool:0")
    feed_dict = {in2: prepro_image}
    features = sess.run(outfinal, feed_dict=feed_dict)[0][0]
    feed_dict = {graph.get_tensor_by_name("decoder/Input_Features:0"): features}
    sentence = []
    for i,outs in enumerate(outputs):
        sentence.append(graph.get_tensor_by_name("decoder/"+outs+":0"))
    prob = sess.run(sentence, feed_dict=feed_dict)
    return " ".join(IDs_to_Words(prob)).split("</S>")[0]

In [6]:
sess = init_caption_generator()

In [7]:
%timeit generate_caption(sess, "../Images/3.jpg")

The slowest run took 17.49 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 192 ms per loop


In [8]:
generate_caption(sess, "../Images/3.jpg")

u'a man is standing on a grassy hill overlooking a lake . '